<a href="https://colab.research.google.com/github/thegregbeyond/FreeFuse-AI-Calbright-Project/blob/main/Video_Face_Blurring_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libs
!pip install insightface onnxruntime-gpu -q

# Import libs
import os
from pathlib import Path
import cv2
import numpy as np
from insightface.app import FaceAnalysis

# --- User Params ---

INPUT_FOLDER        = "/content/drive/MyDrive/FreeFuse_Project/Videos/Input"
OUTPUT_FOLDER       = "/content/drive/MyDrive/FreeFuse_Project/Videos/Blurred"
BLUR_INTENSITY      = 51        # must be odd
FACE_CONF_THRESHOLD = 0.3       # detection score threshold (0–1)

# --- End Parameters ---

# Initialize face detector
print("Loading RetinaFace via FaceAnalysis…")
fa = FaceAnalysis(allowed_modules=["detection"], providers=["CPUExecutionProvider"])
fa.prepare(ctx_id=0, det_size=(640, 640))

def blur_roi(frame, x1, y1, x2, y2, ksize):"
    roi = frame[y1:y2, x1:x2]
    if roi.size:
        frame[y1:y2, x1:x2] = cv2.GaussianBlur(roi, (ksize, ksize), 0)

# Iterate over each video in the input folder
for vid_path in Path(INPUT_FOLDER).glob("*.*"):
    if vid_path.suffix.lower() not in [".mp4", ".mov", ".avi", ".mkv"]:
        continue

    cap = cv2.VideoCapture(str(vid_path))
    if not cap.isOpened():
        print(f"Couldn’t open {vid_path.name}, skipping.")
        continue

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out_path = Path(OUTPUT_FOLDER) / f"{vid_path.stem}_blurred{vid_path.suffix}"
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(out_path), fourcc, fps, (w, h))

    print(f"\nProcessing {vid_path.name} → {out_path.name}")
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # detect faces
        faces = fa.get(frame)
        for face in faces:
            score = face.det_score
            if score < FACE_CONF_THRESHOLD:
                continue
            x1, y1, x2, y2 = map(int, face.bbox)
            blur_roi(frame, x1, y1, x2, y2, BLUR_INTENSITY)

        writer.write(frame)
        frame_idx += 1
        if frame_idx % 100 == 0:
            print(f"   processed {frame_idx}/{total} frames…")

    cap.release()
    writer.release()
    print(f"Finished {out_path.name}")

print("\nAll videos processed!")

Loading RetinaFace via FaceAnalysis…
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 85065.76KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)

▶ Processing 3249672-uhd_3840_2160_25fps.mp4 → 3249672-uhd_3840_2160_25fps_blurred.mp4
   processed 100/204 frames…
   proces